In [1]:
import ultralytics
import cv2
import pandas as pd
import numpy as np
ultralytics.checks()

Ultralytics YOLOv8.0.96  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 176.7/237.3 GB disk)


In [2]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [3]:
model.train(data='web-min-project-akhir.v3i.yolov8\data.yaml',epochs=100,batch=5,imgsz=640,patience=5,optimizer='Adam',device=0,pretrained=True)

New https://pypi.org/project/ultralytics/8.0.111 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.96  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=web-min-project-akhir.v3i.yolov8\data.yaml, epochs=100, patience=5, batch=5, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, re


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/100     0.893G      1.864      1.492      1.604         61        640: 100%|██████████| 75/75 [00:17<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747      0.606      0.556      0.561       0.29

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      8/100      0.96G      1.874      1.474      1.587         84        640: 100%|██████████| 75/75 [00:17<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747      0.763      0.583      0.658      0.345

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      9/100      1.02G      1.812      1.363      1.536         81        640: 100%|██████████| 75/75 [00

WARNING  ConfusionMatrix plot failure: Invalid format specifier '%d' for object of type 'float'



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     16/100         1G      1.691      1.225      1.464        192        640: 100%|██████████| 75/75 [00:18<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747       0.81       0.61      0.708      0.388

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     17/100     0.977G      1.711      1.231      1.493         70        640: 100%|██████████| 75/75 [00:18<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747      0.789      0.618        0.7      0.365

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     18/100      1.03G      1.695      1.259      1.488         69        640: 100%|██████████| 75/75 [00

     36/100     0.925G      1.576      1.079      1.415         73        640: 100%|██████████| 75/75 [00:18<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747       0.76      0.732      0.757      0.422

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     37/100     0.996G      1.568      1.079       1.43         66        640: 100%|██████████| 75/75 [00:17<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<0
                   all         47        747      0.821      0.721      0.803      0.457

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     38/100     0.925G      1.584      1.069       1.41         93        640: 100%|██████████| 75/75 [00:17<00:00,  4.
                 Class     Images  Instances      Box(P         

WARNING  ConfusionMatrix plot failure: Invalid format specifier '%d' for object of type 'float'



38 epochs completed in 0.208 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics YOLOv8.0.96  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<0
                   all         47        747      0.793       0.73      0.816       0.46
                   car         47        282      0.843      0.809      0.887      0.544
             crosswalk         47        113      0.742       0.77      0.835      0.514
            motorcycle         47        352      0.795      0.611      0.727      0.321
Speed: 0.9ms preprocess, 9.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\detect\train4


WARNING  ConfusionMatrix plot failure: Invalid format specifier '%d' for object of type 'float'


In [4]:
model.val()

Ultralytics YOLOv8.0.96  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning E:\MyProject\YOLOv8\datasets\web-min-project-akhir.v3i.yolov8\valid\labels.cache... 47 images, 0 backgrou
WARNING  Box and segment counts should be equal, but got len(segments) = 30, len(boxes) = 747. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07
                   all         47        747      0.795      0.726      0.816      0.461
                   car         47        282      0.844      0.804      0.887      0.544
             crosswalk         47        113      0.741       0.77      0.834      0.517
            motorcycle         47        35

WARNING  ConfusionMatrix plot failure: Invalid format specifier '%d' for object of type 'float'


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x0000018AE954B750>
fitness: 0.4964071060089821
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.54364,     0.51693,     0.32214])
names: {0: 'car', 1: 'crosswalk', 2: 'motorcycle'}
plot: True
results_dict: {'metrics/precision(B)': 0.795112482762673, 'metrics/recall(B)': 0.726259097314457, 'metrics/mAP50(B)': 0.8159508542165032, 'metrics/mAP50-95(B)': 0.46090224509703526, 'fitness': 0.4964071060089821}
save_dir: WindowsPath('runs/detect/val4')
speed: {'preprocess': 1.2849848321143618, 'inference': 14.296602695546252, 'loss': 0.0, 'postprocess': 7.429046833768804}

In [5]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + 0.25*(w-x))
            cy = (y + 0.75*(h-y))

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [6]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

In [7]:
class_list=['car','crosswalk','motorcycle']

In [8]:
tracker=Tracker()

In [9]:
from IPython.display import Image

In [10]:
frame2=cv2.imread('Desktop Screenshot 2023.05.31 - 17.23.45.06.png')
frame2=cv2.resize(frame2,(1920,1080))

In [11]:
result=model.predict(frame2)
a=result[0].boxes.boxes.cpu().numpy()
px = pd.DataFrame(a).astype('float')
list_=[]
for index,row in px.iterrows():
    x1=int(row[0])
    y1=int(row[1])
    x2=int(row[2])
    y2=int(row[3])
    d=int(row[5])
    c=class_list[d]
    if d==0 or d==2:
        list_.append([x1,y1,x2,y2])
bbox_id=tracker.update(list_)
for bbox in bbox_id:
    x3,y3,x4,y4,id=bbox
    cx=int(x3+0.25*(x4-x3))
    cy=int(y3+0.75*(y4-y3))
    cv2.circle(frame2,(cx,cy),4,(0,0,255),-1)
    cv2.putText(frame2,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
cv2.imshow("RGB",frame2)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 3 cars, 1 crosswalk, 3 motorcycles, 40.2ms
Speed: 1.5ms preprocess, 40.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
